In [1]:
# 필요한 라이브러리 Import
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime
from typing import List, Dict, Optional

In [2]:
# Few-shot 예시 정의
examples = [
    {
        "movie": "탑건",
        "emoji": "🛩️👨‍✈️🔥",
        "explanation": "비행기(action), 파일럿(캐릭터), 불타는 액션(분위기)",
        "genre": "액션, 드라마"
    },
    {
        "movie": "대부",
        "emoji": "👨‍👨‍👦🔫🍝",
        "explanation": "가족(테마), 총기(범죄), 이탈리안 음식(문화)",
        "genre": "범죄, 드라마"
    },
    {
        "movie": "타이타닉",
        "emoji": "🚢💑❄️",
        "explanation": "배(배경), 로맨스(장르), 차가운 바다(결말)",
        "genre": "로맨스, 드라마"
    },
    {
        "movie": "쥬라기 공원",
        "emoji": "🦖🏃💨",
        "explanation": "공룡(주제), 도망치는 인간(액션), 긴박감(분위기)",
        "genre": "SF, 어드벤처"
    }
]

In [3]:
# 향상된 메모리 관리 클래스
class EnhancedMovieMemory:
    def __init__(self):
        self.memory = ConversationBufferMemory(
            return_messages=True,
            memory_key="history"
        )
        self.movie_history: List[Dict] = []

    def add_movie(self, movie: str, emojis: str, explanation: str, genre: str):
        """영화 정보를 저장"""
        self.movie_history.append({
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "movie": movie,
            "emojis": emojis,
            "explanation": explanation,
            "genre": genre
        })

    def get_movie_info(self, movie: str) -> Optional[Dict]:
        """특정 영화 정보 검색"""
        for entry in self.movie_history:
            if entry["movie"].lower() == movie.lower():
                return entry
        return None

    def get_all_movies(self) -> List[Dict]:
        """모든 영화 정보 반환"""
        return self.movie_history

    def get_movies_by_genre(self, genre: str) -> List[Dict]:
        """장르별 영화 검색"""
        return [entry for entry in self.movie_history if genre.lower() in entry["genre"].lower()]

In [4]:
# 프롬프트 템플릿 생성
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "영화 '{movie}'를 3개의 이모티콘으로 표현해주세요."),
    ("assistant", "이 영화를 세 개의 이모티콘으로 표현하면 {emoji}입니다.\n{explanation}\n장르: {genre}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 영화를 3개의 이모티콘으로 표현하는 전문가입니다. 
    각 이모티콘은 영화의 핵심 요소(캐릭터, 장르, 분위기, 주제 등)를 나타내야 합니다.
    응답에는 반드시 영화의 장르도 포함해야 합니다."""),
    MessagesPlaceholder(variable_name="history"),
    few_shot_prompt,
    ("human", "영화 '{input}'를 3개의 이모티콘으로 표현해주세요."),
])


In [5]:

# 메모리 확인용 프롬프트
memory_check_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 이전 대화 기록을 분석하는 어시스턴트입니다.
    대화 기록을 확인하고, 다음 형식으로 정리해주세요:
    
    === 시간순 영화 목록 ===
    1. [시간] {첫 번째 영화} - {이모티콘}
       장르: {장르}
       설명: {설명}
    
    2. [시간] {두 번째 영화} - {이모티콘}
       장르: {장르}
       설명: {설명}
    ..."""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{query}")
])

In [6]:
# 모델과 메모리 초기화
model = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.7)
enhanced_memory = EnhancedMovieMemory()

/var/folders/7p/6bdnyyl97jq1l0vwbrmy3k600000gn/T/ipykernel_48407/3811652569.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferMemory(


In [7]:
# LCEL 체인 구성
chain = (
    RunnablePassthrough.assign(
        history=lambda x: enhanced_memory.memory.load_memory_variables({})["history"]
    )
    | final_prompt 
    | model
)

memory_chain = (
    RunnablePassthrough.assign(
        history=lambda x: enhanced_memory.memory.load_memory_variables({})["history"]
    )
    | memory_check_prompt
    | ChatOpenAI(temperature=0)
)


In [8]:
# 실행 함수들
def get_movie_emojis(movie_title: str):
    """영화 이모티콘 생성"""
    response = chain.invoke({"input": movie_title})
    
    # 응답 파싱 및 저장
    content = response.content
    emojis = content.split("이모티콘으로 표현하면 ")[1].split("입니다.")[0]
    explanation = content.split("\n")[1] if "\n" in content else ""
    genre = content.split("장르: ")[1] if "장르: " in content else "알 수 없음"
    
    # 메모리에 저장
    enhanced_memory.add_movie(movie_title, emojis, explanation, genre)
    enhanced_memory.memory.save_context(
        {"input": f"영화 '{movie_title}'를 3개의 이모티콘으로 표현해주세요."},
        {"output": response.content}
    )
    
    return response.content

def check_memory(query: str = "지금까지 어떤 영화들을 분석했나요?"):
    """메모리 확인"""
    if not enhanced_memory.get_all_movies():
        return "아직 어떤 영화도 분석하지 않았습니다."
    
    response = memory_chain.invoke({"query": query})
    return response.content

def search_movie(movie_title: str):
    """특정 영화 검색"""
    movie_info = enhanced_memory.get_movie_info(movie_title)
    if not movie_info:
        return f"'{movie_title}' 영화에 대한 정보를 찾을 수 없습니다."
    
    return f"""
=== {movie_info['movie']} ===
시간: {movie_info['timestamp']}
이모티콘: {movie_info['emojis']}
설명: {movie_info['explanation']}
장르: {movie_info['genre']}
"""

def search_by_genre(genre: str):
    """장르별 영화 검색"""
    movies = enhanced_memory.get_movies_by_genre(genre)
    if not movies:
        return f"'{genre}' 장르의 영화를 찾을 수 없습니다."
    
    result = f"\n=== {genre} 장르 영화 목록 ===\n"
    for movie in movies:
        result += f"\n{movie['movie']} - {movie['emojis']}"
    return result

In [9]:
print("첫 번째 영화:")
print(get_movie_emojis("매트릭스"))

첫 번째 영화:
이 영화를 세 개의 이모티콘으로 표현하면 💊🔫💻입니다.
약(선택), 총기(액션), 컴퓨터(주제)
장르: 액션, SF


In [10]:
print("\n두 번째 영화:")
print(get_movie_emojis("인셉션"))


두 번째 영화:
이 영화를 세 개의 이모티콘으로 표현하면 🌌💤🌀입니다.
꿈(주제), 잠(상태), 소용돌이(복잡한 플롯)
장르: SF, 스릴러


In [11]:
print("\n세 번째 영화:")
print(get_movie_emojis("해리 포터"))


세 번째 영화:
이 영화를 세 개의 이모티콘으로 표현하면 ⚡️🧙‍♂️🏰입니다.  
번개(해리 포터의 상징), 마법사(캐릭터), 성(호그와트)  
장르: 판타지, 어드벤처


In [12]:
print("\n네 번째 영화:")
print(get_movie_emojis("반지의 제왕"))


네 번째 영화:
이 영화를 세 개의 이모티콘으로 표현하면 💍🧙‍♂️⚔️입니다.  
반지(주제), 마법사(캐릭터), 검(전투)  
장르: 판타지, 어드벤처


In [13]:
print("\n특정 영화 검색:")
print(search_movie("매트릭스"))


특정 영화 검색:

=== 매트릭스 ===
시간: 2024-12-11 09:59:42
이모티콘: 💊🔫💻
설명: 약(선택), 총기(액션), 컴퓨터(주제)
장르: 액션, SF



In [14]:
print("\n장르별 검색:")
print(search_by_genre("SF"))


장르별 검색:

=== SF 장르 영화 목록 ===

매트릭스 - 💊🔫💻
인셉션 - 🌌💤🌀
